In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import NMF;
from sklearn.preprocessing import normalize;
import pickle
import re, string
%matplotlib notebook

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/web-log-dataset/weblog.csv')
df.head(10)

In [ ]:
def clean_text(text):
    
    PUNCTUATION = """!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""    

    text = " ".join([c for c in text if c not in PUNCTUATION])
    
    text = re.sub(r"/"," ",text)
    return text

In [ ]:
from nltk.tokenize import word_tokenize
df['URL'] = df['URL'].apply(word_tokenize)
df['text'] = df['URL'].apply(clean_text)
df.head()

In [ ]:
sentences = [''.join(text) for text in df['text']]

In [ ]:
vectorizer = CountVectorizer(analyzer='word', max_features=1000)
x_counts = vectorizer.fit_transform(sentences)

In [ ]:
x_counts

In [ ]:
transformer = TfidfTransformer(smooth_idf=False);
x_tfidf = transformer.fit_transform(x_counts);

In [ ]:
xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)

In [ ]:
#number of topics
num_topics=5
#obtain a NMF model.
model = NMF(n_components=num_topics, init='nndsvd');
#fit the model
model.fit(xtfidf_norm)

In [ ]:
def get_nmf_topics(model, n_top_words):
    
    #the word ids obtained need to be reverse-mapped to the words so we can print the topic names.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(num_topics):
        
        #for each topic, obtain the largest values, and add the words they map to into the dictionary.
        words_ids = model.components_[i].argsort()[:-n_top_words - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Tarea # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict)

In [ ]:
get_nmf_topics(model, 10)